In [1]:
from typing import *
import wafel
import ctypes as C
import struct
from random import *
import numpy as np
import copy
import numpy as np

In [2]:
%run utils.ipynb

In [3]:
def get_fitness(game,
                game_vars_match_exact, game_vars_match_range, game_vars_approximate_weighted):
    fitness = 0
    success = True
    for var in game_vars_match_exact:
        if game.read(var) != game_vars_match_exact[var]:
            #note: if your weights/possible changes in values for the weighted
            #approximate variables are pretty small, you may need to
            #reduce this penalty so those changes aren't lost in float rounding if you
            #care about other changes even when one of the exact matches fails.
            fitness += 1e10
    for var in game_vars_match_range:
        val = game.read(var)
        goal = game_vars_match_range[var][0]
        tol = game_vars_match_range[var][1]
        if abs(val - goal) > tol:
            fitness += min(1e6*(abs(val - goal) - tol)/tol, 1e9)
    for var in game_vars_approximate_weighted:
        val = game.read(var)
        fitness += abs(val - game_vars_approximate_weighted[var][0])*game_vars_approximate_weighted[var][1]
    return fitness

In [4]:
def l1_of_diff(m64):
    tot = 0
    for i in range(len(m64) - 1):
        tot += abs(m64[i].stick_x - m64[i+1].stick_x) + abs(m64[i].stick_y - m64[i+1].stick_y)
    return tot

#Special version of the annealing function for resyncing.
#Has some bells and whistles.
def anneal_for_resync(game,
                      m64,
                      starting_savestate,
                      start_frame, end_frame,
                      game_vars_match_exact, game_vars_match_range, game_vars_approximate_weighted,
                      weighted_sum_acceptable,
                      max_iterations,
                      starting_temperature,
                      l1_diff_penalty_strength = 0.0):
    game.load_state(starting_savestate)
    
    
    for f in range(start_frame, end_frame):
        set_inputs(game, m64[f])
        game.advance()
    
    #Since the camera may be different, the newest
    #frame to sync could be way off from what we want.
    #Therefore, start by doing a search of all possible joysticks for the last frame.
    #Takes a long time, so I go in coarser intervals. Annealing should be
    #able to find its way into a better granularity.
    best_fitness = 1e15
    best_input = None
    before_final_step = game.save_state()
    for x in range(0, 256, 4):
        for y in range(0, 256, 4):
            game.load_state(before_final_step)
            new_input = wafel.Input(buttons = m64[end_frame].buttons, stick_x = x, stick_y = y)
            set_inputs(game, new_input)
            game.advance()
            fitness = get_fitness(game, game_vars_match_exact, game_vars_match_range, game_vars_approximate_weighted)
            if l1_diff_penalty_strength > 0:
                fitness += l1_diff_penalty_strength*l1_of_diff(m64[start_frame:end_frame+1])
            if fitness < best_fitness:
                best_fitness = fitness
                best_input = new_input
    m64[end_frame] = best_input
    
    
    print('*************************************************************')
    print('BEFORE ANNEALING')
    print('Frame {0}: fitness is {1}'.format(end_frame, best_fitness))
    game.load_state(before_final_step)
    set_inputs(game, best_input)
    game.advance()
    print('Exact matches:')
    for var in game_vars_match_exact:
        print('{0}: wanted {1}, got {2}'.format(var, game_vars_match_exact[var], game.read(var)))
    print('Matches within an interval:')
    for var in game_vars_match_range:
        print('{0}: wanted {1}, tolerance {2}, got {3}'.format(var, game_vars_match_range[var][0],
                                                               game_vars_match_range[var][1], game.read(var)))
    print('Weighted approximations:')
    for var in game_vars_approximate_weighted:
        print('{0}: wanted {1}, weight {2}, got {3}'.format(var, game_vars_approximate_weighted[var][0],
                                                            game_vars_approximate_weighted[var][1], game.read(var)))
        
    
    
    #Now do the annealing step
    #Not going to bother with random restarts here, though it is a plausible improvement.
    #Probably better for now to just go with very low temperature since we need to do faster
    #searches. Therefore default starting temperature is 0. Plausible to use nonzero ones but
    #they should be pretty small unless you're willing to wait a long time or you want to
    #do random restarts.
    orig_m64 = copy.copy(m64)
    best_m64 = copy.copy(m64)
    
    cur_fitness = best_fitness
    
    #Bunch of parameters here to play with
    #Might want to make them function arguments at some point
    many_change_prob = .1
    big_change_prob = .3
    max_changes = 5
    max_size = 30
    
    #Weight the distribution to preferentially choose the frames toward the end
    #I'm guessing this will help at least somewhat
    possible_frames = np.arange(start_frame, end_frame + 1)
    exp_decay = np.exp(-np.arange(end_frame - start_frame, -1, -1)*(1.0/(start_frame-end_frame+1)))
    exp_decay = exp_decay/(exp_decay.sum())
    
    temp = starting_temperature
    
    for it in range(max_iterations):
        if cur_fitness <= weighted_sum_acceptable:
            break
        if it % 300 == 0:
            temp *= .97
        changed_frames = []
        num_changes = randint(1, max_changes)
        if random() > many_change_prob:
            num_changes = 1
        change_frame_nums = np.random.choice(possible_frames, size = num_changes, p = exp_decay)
        for change_frame_num in change_frame_nums:
            #change_frame = randint(start_frame, end_frame)
            changed_frames.append((change_frame_num, m64[change_frame_num]))
            change_size = randint(1, max_size)
            if random() > big_change_prob:
                change_size = 1
            change_dir = randint(0, 3)
            frame_inp = m64[change_frame_num]
            new_inp = wafel.Input(frame_inp.buttons, frame_inp.stick_x, frame_inp.stick_y)
            m64[change_frame_num] = new_inp
            for inc in range(change_size):
                if change_dir == 0:
                    new_inp.stick_x = increment_coord(new_inp.stick_x)
                elif change_dir == 1:
                    new_inp.stick_y = increment_coord(new_inp.stick_y)
                elif change_dir == 2:
                    new_inp.stick_x = 255 - increment_coord(255 - new_inp.stick_x)
                else:
                    new_inp.stick_y = 255 - increment_coord(255 - new_inp.stick_y)
        game.load_state(starting_savestate)
        for frame in range(start_frame, end_frame + 1):
            set_inputs(game, m64[frame])
            game.advance()
        fitness = get_fitness(game, game_vars_match_exact, game_vars_match_range, game_vars_approximate_weighted)
        if l1_diff_penalty_strength > 0:
            fitness += l1_diff_penalty_strength*l1_of_diff(m64[start_frame:end_frame+1])
        
        if fitness < cur_fitness:
            cur_fitness = fitness
            if fitness < best_fitness:
                best_fitness = fitness
                best_m64 = copy.copy(m64)
        elif temp > 0 and random() < np.exp(-(fitness - cur_fitness) / temp):
            cur_fitness = fitness
        else:
            for changed_frame_info in changed_frames[::-1]:
                m64[changed_frame_info[0]] = changed_frame_info[1]
    
    for i in range(len(best_m64)):
        m64[i] = best_m64[i]
    game.load_state(starting_savestate)
    for frame in range(start_frame, end_frame + 1):
        set_inputs(game, m64[frame])
        game.advance()
    
    print('*************************************************************')
    print('AFTER ANNEALING')
    print('Frame {0}: fitness is {1}'.format(end_frame, best_fitness))
    print('Exact matches:')
    for var in game_vars_match_exact:
        print('{0}: wanted {1}, got {2}'.format(var, game_vars_match_exact[var], game.read(var)))
    print('Matches within an interval:')
    for var in game_vars_match_range:
        print('{0}: wanted {1}, tolerance {2}, got {3}'.format(var, game_vars_match_range[var][0],
                                                               game_vars_match_range[var][1], game.read(var)))
    print('Weighted approximations:')
    for var in game_vars_approximate_weighted:
        print('{0}: wanted {1}, weight {2}, got {3}'.format(var, game_vars_approximate_weighted[var][0],
                                                            game_vars_approximate_weighted[var][1], game.read(var)))
    
    
    
    return best_fitness

In [5]:
#Tries to sync up filename_toresync to filename_original and saves the result to output_filename
#For each variable of interest, it tries to find joystick values in filename_new that
#will match up variables.
#game_vars_match_exact, if not None, is a list of variable strings which
#the function will try to sync exactly.
#game_vars_match_range, if not None, is a dictionary from variable strings
#to tolerances which define the amount by which that variable can be off
#game_vars_approximate_weighted, if not None, is a dictionary from variable strings
#to weights. The function will attempt to minimize
#sum(abs(original_value-new_value)*weight) given that the matching variables
#all have acceptable values.
#It is permissible for variables to apear both in the weighted sum and in the matching.
#This may be desirable if e.g. it's necessary to have yaw in a certain interval and we
#also want to get it close to the center of that interval.
#The function tries to match 1 frame, then 2, etc. If the matching variables are
#all acceptable and the sum of the weighted errors is at most weighted_sum_acceptable,
#the function will continue to the next frame.
#Otherwise, it will try max_iteration times.
#To sync up at the very beginning, starts some frames back. By default, it goes back 20
def resync_tas(game,
               filename_original, filename_toresync, filename_output,
               start_frame_original, end_frame_original, start_frame_toresync,
               game_vars_match_exact = None,
               game_vars_match_range = None,
               game_vars_approximate_weighted = None,
               weighted_sum_acceptable = 0.0,
               frames_back = 20,
               max_iterations = 3000,
               starting_temperature = 0.0,
               l1_diff_penalty_strength = 0.0):
    
    power_on = game.save_state()
    m64_original = wafel.load_m64(filename_original)
    m64_toresync = wafel.load_m64(filename_toresync)
    
    relevant_variables = set([var for var in game_vars_match_exact] +
                             [var for var in game_vars_match_range] +
                             [var for var in game_vars_approximate_weighted])
    #First, find values of the variables in the original TAS.
    observed_values = {var: [] for var in relevant_variables}
    #Advance to start of resync window
    for f in range(start_frame_original):
        set_inputs(game, m64_original[1][f])
        game.advance()
    #Now record values
    for f in range(end_frame_original - start_frame_original + 1):
        set_inputs(game, m64_original[1][f + start_frame_original])
        game.advance()
        for var in relevant_variables:
            observed_values[var].append(game.read(var))
            
    
    #Now start working on the output TAS.
    #Use the start of the new one and the end of the old one.
    #Then anneal to try to match values.
    m64_output = m64_toresync[1][:start_frame_toresync] + m64_original[1][start_frame_original:]
    #First, try to resync initial conditions.
    game.load_state(power_on)
    start_frame_anneal = start_frame_toresync - frames_back
    for f in range(start_frame_anneal):
        set_inputs(game, m64_output[f])
        game.advance()
    savestate_before_initial_conditions = game.save_state()
    frames_resyncing_count = 0
    while frames_resyncing_count < end_frame_original - start_frame_original + 1:
        result = 1e10
        retries = 0
        while result >= 400:
            if retries > 0:
                print('RETRY {0}'.format(retries))
            m64_output_prev = copy.copy(m64_output)
            result = anneal_for_resync(game,
                              m64_output,
                              savestate_before_initial_conditions,
                              start_frame_anneal + frames_resyncing_count, start_frame_anneal + frames_back + frames_resyncing_count,
                              {var: observed_values[var][frames_resyncing_count] for var in game_vars_match_exact},
                              {var: (observed_values[var][frames_resyncing_count], game_vars_match_range[var]) for var in game_vars_match_range},
                              {var: (observed_values[var][frames_resyncing_count], game_vars_approximate_weighted[var]) for var in game_vars_approximate_weighted},
                              weighted_sum_acceptable = weighted_sum_acceptable,
                              max_iterations = max_iterations*((frames_resyncing_count == 0) * 10 + 1 + retries), #Take extra effort to resync the start
                              starting_temperature = starting_temperature*(5**retries),
                              l1_diff_penalty_strength = l1_diff_penalty_strength)
            retries += 1
            #if result >= 10000:
            #    m64_output = m64_output_prev
            if retries > 20:
                break
        game.load_state(savestate_before_initial_conditions)
        set_inputs(game, m64_output[start_frame_anneal + frames_resyncing_count])
        game.advance()
        savestate_before_initial_conditions = game.save_state()
        frames_resyncing_count += 1
        wafel.save_m64(filename_output, m64_toresync[0], m64_output)
    

In [6]:
# resync_tas(game,
#                filename_original, filename_toresync, filename_output,
#                start_frame_original, end_frame_original, start_frame_toresync,
#                game_vars_match_exact = None,
#                game_vars_match_range = None,
#                game_vars_approximate_weighted = None,
#                weighted_sum_acceptable = 0.0,
#                frames_back = 20,
#                max_iterations = 10000,
#                starting_temperature = 0.0):

In [7]:
# game = wafel.Game('sm64_jp.dll')
# main_folder = 'C://Users/David/Documents/Miscellaniety/Mupen/abc70/resyncing_after_wf_updates/'

# filename_original = main_folder+'wfreds_redid_misaclimb.m64'
# filename_toresync = main_folder+'workspace.m64'
# filename_output = main_folder+'auto_resynced_redsclimb.m64'

# game_vars_match_exact = ['gMarioState.action']
# game_vars_match_range = {'gMarioState.pos[0]': 1.5,
#                          'gMarioState.pos[1]': 20,
#                          'gMarioState.pos[2]': 1.5,
#                          'gMarioState.faceAngle[1]': 512,
#                          'gMarioState.forwardVel': .15}
# game_vars_approximate_weighted = {'gMarioState.pos[0]': 15.0,
#                                   'gMarioState.pos[1]': 1.0,
#                                   'gMarioState.pos[2]': 15.0,
#                                   'gMarioState.faceAngle[1]': 0.3,
#                                   'gMarioState.forwardVel': 30.0
#                                   }

# resync_tas(game,
#            filename_original, filename_toresync, filename_output,
#            28455, 28570, 29923,
#            game_vars_match_exact = game_vars_match_exact,
#            game_vars_match_range = game_vars_match_range,
#            game_vars_approximate_weighted = game_vars_approximate_weighted,
#            starting_temperature = 1.5)


In [8]:
#Jong wf to bitdw resync
# game = wafel.Game('sm64_jp.dll')
# main_folder = 'D://David/Mupen/abc70/resync_wf_after_new_100tower/'

# #filename_original = main_folder+'wfreds_redid_misaclimb.m64'
# filename_original = main_folder+'WIP_TO_HAUNTED_BOOKS.m64'
# filename_toresync = main_folder+'put_in_jong_to_bitdw.m64'
# filename_output = main_folder+'resynced_jong_to_bitdw_notrngyet.m64'

# game_vars_match_exact = ['gMarioState.action']
# game_vars_match_range = {'gMarioState.pos[0]': .8,
#                          'gMarioState.pos[1]': 90,
#                          'gMarioState.pos[2]': .8,
#                          'gMarioState.faceAngle[1]': 512,
#                          'gMarioState.forwardVel': .2,
#                          'gMarioState.angleVel[1]': 128}
# game_vars_approximate_weighted = {'gMarioState.pos[0]': 15.0,
#                                   'gMarioState.pos[1]': 1.0,
#                                   'gMarioState.pos[2]': 15.0,
#                                   'gMarioState.faceAngle[0]': 0.3,
#                                   'gMarioState.faceAngle[1]': 0.3,
#                                   'gMarioState.forwardVel': 30.0,
#                                   'gMarioState.angleVel[1]': 1.0
#                                   }

# resync_tas(game,
#            filename_original, filename_toresync, filename_output,
#            34663, 34766, 34660,
#            game_vars_match_exact = game_vars_match_exact,
#            game_vars_match_range = game_vars_match_range,
#            game_vars_approximate_weighted = game_vars_approximate_weighted,
#            starting_temperature = 1.5,
#            weighted_sum_acceptable = 1.0)

In [9]:
#MHMCM RESYNC
# game = wafel.Game('sm64_jp.dll')
# main_folder = 'D://David/Mupen/abc70/resync_wf_after_new_100tower/'

# #filename_original = main_folder+'wfreds_redid_misaclimb.m64'
# filename_original = main_folder+'old_wip.m64'
# filename_toresync = main_folder+'synced_position_mhmcm.m64'
# filename_output = main_folder+'resynced_mhmcm.m64'

# game_vars_match_exact = ['gMarioState.action']
# game_vars_match_range = {'gMarioState.pos[0]': .8,
#                          'gMarioState.pos[1]': 90,
#                          'gMarioState.pos[2]': .8,
#                          'gMarioState.faceAngle[1]': 512,
#                          'gMarioState.forwardVel': .2,
#                          'gMarioState.angleVel[1]': 128}
# game_vars_approximate_weighted = {'gMarioState.pos[0]': 15.0,
#                                   'gMarioState.pos[1]': 1.0,
#                                   'gMarioState.pos[2]': 15.0,
#                                   'gMarioState.faceAngle[0]': 0.3,
#                                   'gMarioState.faceAngle[1]': 0.3,
#                                   'gMarioState.forwardVel': 30.0,
#                                   'gMarioState.angleVel[1]': 1.0
#                                   }

# resync_tas(game,
#            filename_original, filename_toresync, filename_output,
#            58563, 59487, 55643,
#            game_vars_match_exact = game_vars_match_exact,
#            game_vars_match_range = game_vars_match_range,
#            game_vars_approximate_weighted = game_vars_approximate_weighted,
#            starting_temperature = 1.5,
#            frame_back = 45,
#            weighted_sum_acceptable = 1.0)

In [10]:
# game = wafel.Game('sm64_jp.dll')
# main_folder = 'D://David/Mupen/abc70/resync_wf_after_new_100tower/'

# #filename_original = main_folder+'wfreds_redid_misaclimb.m64'
# filename_original = main_folder+'old_wip.m64'
# filename_toresync = main_folder+'do_cannonless_goodrng.m64'
# filename_output = main_folder+'auto_resynced_cannonless.m64'

# game_vars_match_exact = ['gMarioState.action',
#                          'gMarioState.numStars']
# game_vars_match_range = {'gMarioState.pos[0]': .8,
#                          'gMarioState.pos[1]': 90,
#                          'gMarioState.pos[2]': .8,
#                          'gMarioState.faceAngle[0]': 512,
#                          'gMarioState.faceAngle[1]': 512,
#                          'gMarioState.forwardVel': .5}
# game_vars_approximate_weighted = {'gMarioState.pos[0]': 15.0,
#                                   'gMarioState.pos[1]': 1.0,
#                                   'gMarioState.pos[2]': 15.0,
#                                   'gMarioState.faceAngle[1]': 0.3,
#                                   'gMarioState.forwardVel': 30.0
#                                   }

# resync_tas(game,
#            filename_original, filename_toresync, filename_output,
#            34065, 34180, 33760,
#            game_vars_match_exact = game_vars_match_exact,
#            game_vars_match_range = game_vars_match_range,
#            game_vars_approximate_weighted = game_vars_approximate_weighted,
#            starting_temperature = 1.5)


In [13]:
game = wafel.Game('sm64_jp.dll')
main_folder = 'C://Users/David/Documents/Mupen/wafel_bruteforce/wafel_bruteforce/wafel_081_tools/m64s/'

#Try to resync ttm monkey with water upwarp
filename_original = main_folder+'breathtaking.m64'
filename_toresync = main_folder+'try_full_route.m64'
filename_output = main_folder+'100monkey_upwarp.m64'


game_vars_match_exact = []
game_vars_match_range = {'gMarioState.pos[0]': 3.0,
                         'gMarioState.pos[1]': 3.0,
                         'gMarioState.pos[2]': 3.0,
                         'gMarioState.forwardVel': .1,
                         'gMarioState.faceAngle[1]': 100}
game_vars_approximate_weighted = {'gMarioState.pos[0]': 1.0,
                                  'gMarioState.pos[1]': 1.0,
                                  'gMarioState.pos[2]': 1.0,
                                  'gMarioState.faceAngle[1]': .05,
                                  'gMarioState.faceAngle[0]': .05,
                                  'gMarioState.forwardVel': 100.0}

resync_tas(game,
           filename_original, filename_toresync, filename_output,
           71878, 72077, 72313,
           game_vars_match_exact = game_vars_match_exact,
           game_vars_match_range = game_vars_match_range,
           game_vars_approximate_weighted = game_vars_approximate_weighted,
           starting_temperature = 3.5,
           weighted_sum_acceptable = 1.0,
           frames_back = 60,
           max_iterations = 8000,
           l1_diff_penalty_strength = 0.0)

*************************************************************
BEFORE ANNEALING
Frame 72313: fitness is 73057584.73674266
Exact matches:
Matches within an interval:
gMarioState.pos[0]: wanted 384.45355224609375, tolerance 3.0, got 335.9151611328125
gMarioState.pos[1]: wanted -4332.0, tolerance 3.0, got -4332.0
gMarioState.pos[2]: wanted 5548.056640625, tolerance 3.0, got 5412.61328125
gMarioState.forwardVel: wanted 29.3148250579834, tolerance 0.1, got 30.76282501220703
gMarioState.faceAngle[1]: wanted -32758, tolerance 100, got -32633
Weighted approximations:
gMarioState.pos[0]: wanted 384.45355224609375, weight 1.0, got 335.9151611328125
gMarioState.pos[1]: wanted -4332.0, weight 1.0, got -4332.0
gMarioState.pos[2]: wanted 5548.056640625, weight 1.0, got 5412.61328125
gMarioState.faceAngle[1]: wanted -32758, weight 0.05, got -32633
gMarioState.faceAngle[0]: wanted 0, weight 0.05, got 0
gMarioState.forwardVel: wanted 29.3148250579834, weight 100.0, got 30.76282501220703
****************

*************************************************************
BEFORE ANNEALING
Frame 72316: fitness is 0.9163452148437501
Exact matches:
Matches within an interval:
gMarioState.pos[0]: wanted 384.66827392578125, tolerance 3.0, got 384.6610107421875
gMarioState.pos[1]: wanted -4284.0, tolerance 3.0, got -4284.0
gMarioState.pos[2]: wanted 5414.212890625, tolerance 3.0, got 5414.49755859375
gMarioState.forwardVel: wanted 44.76448440551758, tolerance 0.1, got 44.76424026489258
gMarioState.faceAngle[1]: wanted -32758, tolerance 100, got -32746
Weighted approximations:
gMarioState.pos[0]: wanted 384.66827392578125, weight 1.0, got 384.6610107421875
gMarioState.pos[1]: wanted -4284.0, weight 1.0, got -4284.0
gMarioState.pos[2]: wanted 5414.212890625, weight 1.0, got 5414.49755859375
gMarioState.faceAngle[1]: wanted -32758, weight 0.05, got -32746
gMarioState.faceAngle[0]: wanted 0, weight 0.05, got 0
gMarioState.forwardVel: wanted 44.76448440551758, weight 100.0, got 44.76424026489258
*******

*************************************************************
BEFORE ANNEALING
Frame 72321: fitness is 0.8496734619140625
Exact matches:
Matches within an interval:
gMarioState.pos[0]: wanted 385.77264404296875, tolerance 3.0, got 385.77081298828125
gMarioState.pos[1]: wanted -4284.0, tolerance 3.0, got -4284.0
gMarioState.pos[2]: wanted 5188.1484375, tolerance 3.0, got 5188.33837890625
gMarioState.forwardVel: wanted 45.51252365112305, tolerance 0.1, got 45.51044464111328
gMarioState.faceAngle[1]: wanted -32758, tolerance 100, got -32749
Weighted approximations:
gMarioState.pos[0]: wanted 385.77264404296875, weight 1.0, got 385.77081298828125
gMarioState.pos[1]: wanted -4284.0, weight 1.0, got -4284.0
gMarioState.pos[2]: wanted 5188.1484375, weight 1.0, got 5188.33837890625
gMarioState.faceAngle[1]: wanted -32758, weight 0.05, got -32749
gMarioState.faceAngle[0]: wanted -1536, weight 0.05, got -1536
gMarioState.forwardVel: wanted 45.51252365112305, weight 100.0, got 45.51044464111328
*

*************************************************************
BEFORE ANNEALING
Frame 72326: fitness is 0.2652587890625
Exact matches:
Matches within an interval:
gMarioState.pos[0]: wanted 388.525146484375, tolerance 3.0, got 388.54107666015625
gMarioState.pos[1]: wanted -4411.5625, tolerance 3.0, got -4411.5625
gMarioState.pos[2]: wanted 5037.11328125, tolerance 3.0, got 5037.30615234375
gMarioState.forwardVel: wanted 47.7390251159668, tolerance 0.1, got 47.738460540771484
gMarioState.faceAngle[1]: wanted 32117, tolerance 100, got 32117
Weighted approximations:
gMarioState.pos[0]: wanted 388.525146484375, weight 1.0, got 388.54107666015625
gMarioState.pos[1]: wanted -4411.5625, weight 1.0, got -4411.5625
gMarioState.pos[2]: wanted 5037.11328125, weight 1.0, got 5037.30615234375
gMarioState.faceAngle[1]: wanted 32117, weight 0.05, got 32117
gMarioState.faceAngle[0]: wanted 0, weight 0.05, got 0
gMarioState.forwardVel: wanted 47.7390251159668, weight 100.0, got 47.738460540771484
******

*************************************************************
BEFORE ANNEALING
Frame 72331: fitness is 0.2358245849609375
Exact matches:
Matches within an interval:
gMarioState.pos[0]: wanted 423.7822265625, tolerance 3.0, got 423.78436279296875
gMarioState.pos[1]: wanted -4301.5625, tolerance 3.0, got -4301.5625
gMarioState.pos[2]: wanted 4799.861328125, tolerance 3.0, got 4800.05419921875
gMarioState.forwardVel: wanted 47.84455108642578, tolerance 0.1, got 47.8449592590332
gMarioState.faceAngle[1]: wanted 32117, tolerance 100, got 32117
Weighted approximations:
gMarioState.pos[0]: wanted 423.7822265625, weight 1.0, got 423.78436279296875
gMarioState.pos[1]: wanted -4301.5625, weight 1.0, got -4301.5625
gMarioState.pos[2]: wanted 4799.861328125, weight 1.0, got 4800.05419921875
gMarioState.faceAngle[1]: wanted 32117, weight 0.05, got 32117
gMarioState.faceAngle[0]: wanted 0, weight 0.05, got 0
gMarioState.forwardVel: wanted 47.84455108642578, weight 100.0, got 47.8449592590332
*******

...

*************************************************************
BEFORE ANNEALING
Frame 72478: fitness is 0.2437744140625
Exact matches:
Matches within an interval:
gMarioState.pos[0]: wanted 1731.9560546875, tolerance 3.0, got 1731.9212646484375
gMarioState.pos[1]: wanted -1548.0, tolerance 3.0, got -1548.0
gMarioState.pos[2]: wanted 2824.889892578125, tolerance 3.0, got 2824.680908203125
gMarioState.forwardVel: wanted 13.858783721923828, tolerance 0.1, got 13.858783721923828
gMarioState.faceAngle[1]: wanted 8688, tolerance 100, got 8688
Weighted approximations:
gMarioState.pos[0]: wanted 1731.9560546875, weight 1.0, got 1731.9212646484375
gMarioState.pos[1]: wanted -1548.0, weight 1.0, got -1548.0
gMarioState.pos[2]: wanted 2824.889892578125, weight 1.0, got 2824.680908203125
gMarioState.faceAngle[1]: wanted 8688, weight 0.05, got 8688
gMarioState.faceAngle[0]: wanted 4096, weight 0.05, got 4096
gMarioState.forwardVel: wanted 13.858783721923828, weight 100.0, got 13.858783721923828
****

*************************************************************
BEFORE ANNEALING
Frame 72483: fitness is 0.239166259765625
Exact matches:
Matches within an interval:
gMarioState.pos[0]: wanted 1852.00830078125, tolerance 3.0, got 1851.7977294921875
gMarioState.pos[1]: wanted -1508.0, tolerance 3.0, got -1508.0
gMarioState.pos[2]: wanted 2934.034423828125, tolerance 3.0, got 2934.018798828125
gMarioState.forwardVel: wanted 32.75000762939453, tolerance 0.1, got 32.7498779296875
gMarioState.faceAngle[1]: wanted 8688, tolerance 100, got 8688
Weighted approximations:
gMarioState.pos[0]: wanted 1852.00830078125, weight 1.0, got 1851.7977294921875
gMarioState.pos[1]: wanted -1508.0, weight 1.0, got -1508.0
gMarioState.pos[2]: wanted 2934.034423828125, weight 1.0, got 2934.018798828125
gMarioState.faceAngle[1]: wanted 8688, weight 0.05, got 8688
gMarioState.faceAngle[0]: wanted 4096, weight 0.05, got 4096
gMarioState.forwardVel: wanted 32.75000762939453, weight 100.0, got 32.7498779296875
******

*************************************************************
AFTER ANNEALING
Frame 72487: fitness is 3930547.0936431885
Exact matches:
Matches within an interval:
gMarioState.pos[0]: wanted 1948.071533203125, tolerance 3.0, got 1945.1241455078125
gMarioState.pos[1]: wanted -1512.0, tolerance 3.0, got -1512.0
gMarioState.pos[2]: wanted 3025.339599609375, tolerance 3.0, got 3040.130859375
gMarioState.forwardVel: wanted 33.350013732910156, tolerance 0.1, got 33.250003814697266
gMarioState.faceAngle[1]: wanted 8688, tolerance 100, got 8693
Weighted approximations:
gMarioState.pos[0]: wanted 1948.071533203125, weight 1.0, got 1945.1241455078125
gMarioState.pos[1]: wanted -1512.0, weight 1.0, got -1512.0
gMarioState.pos[2]: wanted 3025.339599609375, weight 1.0, got 3040.130859375
gMarioState.faceAngle[1]: wanted 8688, weight 0.05, got 8693
gMarioState.faceAngle[0]: wanted 4096, weight 0.05, got 4096
gMarioState.forwardVel: wanted 33.350013732910156, weight 100.0, got 33.250003814697266
RETR

*************************************************************
BEFORE ANNEALING
Frame 72487: fitness is 3827017.3110432946
Exact matches:
Matches within an interval:
gMarioState.pos[0]: wanted 1948.071533203125, tolerance 3.0, got 1945.0947265625
gMarioState.pos[1]: wanted -1512.0, tolerance 3.0, got -1512.0
gMarioState.pos[2]: wanted 3025.339599609375, tolerance 3.0, got 3039.820556640625
gMarioState.forwardVel: wanted 33.350013732910156, tolerance 0.1, got 33.25025177001953
gMarioState.faceAngle[1]: wanted 8688, tolerance 100, got 8772
Weighted approximations:
gMarioState.pos[0]: wanted 1948.071533203125, weight 1.0, got 1945.0947265625
gMarioState.pos[1]: wanted -1512.0, weight 1.0, got -1512.0
gMarioState.pos[2]: wanted 3025.339599609375, weight 1.0, got 3039.820556640625
gMarioState.faceAngle[1]: wanted 8688, weight 0.05, got 8772
gMarioState.faceAngle[0]: wanted 4096, weight 0.05, got 4096
gMarioState.forwardVel: wanted 33.350013732910156, weight 100.0, got 33.25025177001953
*****

KeyboardInterrupt: 

In [ ]:
#Try to resync DDD, but ahead 1f?
game = wafel.Game('sm64_us.dll')

filename_original = 'C:/Users/David/Downloads/bowsersub_tjk_lag.m64'
filename_toresync = 'C:/Users/David/Downloads/bowsersub_tjk_lag_resync.m64'
filename_output = 'C:/Users/David/Downloads/attempt_save_1f_bowsersub.m64'

game_vars_match_exact = []
game_vars_match_range = {'gMarioState.pos[0]': 50.0,
                         'gMarioState.pos[1]': 50.0,
                         'gMarioState.pos[2]': 50.0}
game_vars_approximate_weighted = {'gMarioState.pos[0]': 1.0,
                                  'gMarioState.pos[1]': 1.0,
                                  'gMarioState.pos[2]': 1.0,
                                  'gMarioState.faceAngle[1]': .001,
                                  'gMarioState.faceAngle[0]': .01}

resync_tas(game,
           filename_original, filename_toresync, filename_output,
           3690, 4380, 3690,
           game_vars_match_exact = game_vars_match_exact,
           game_vars_match_range = game_vars_match_range,
           game_vars_approximate_weighted = game_vars_approximate_weighted,
           starting_temperature = 0.2,
           weighted_sum_acceptable = 1.0,
           frames_back = 60,
           max_iterations = 6000,
           l1_diff_penalty_strength = .06)

In [ ]:
# game = wafel.Game('sm64_jp.dll')

# filename_original = 'm64s/old_wf100.m64'
# filename_toresync = 'm64s/new_wf100_toresync.m64'
# filename_output = 'm64s/resynced_attempt.m64'

# game_vars_match_exact = ['gMarioState.action',
#                          'gMarioState.numStars']
# game_vars_match_range = {'gMarioState.pos[0]': 1.5,
#                          'gMarioState.pos[1]': .01,
#                          'gMarioState.pos[2]': 1.5,
#                          'gMarioState.faceAngle[1]': 128,
#                          'gMarioState.forwardVel': .07}
# game_vars_approximate_weighted = {'gMarioState.pos[0]': 15.0,
#                                   'gMarioState.pos[1]': 1.0,
#                                   'gMarioState.pos[2]': 15.0,
#                                   'gMarioState.faceAngle[1]': 0.3,
#                                   'gMarioState.forwardVel': 45.0
#                                   }

# resync_tas(game,
#            filename_original, filename_toresync, filename_output,
#            29192, 29450, 28911,
#            game_vars_match_exact = game_vars_match_exact,
#            game_vars_match_range = game_vars_match_range,
#            game_vars_approximate_weighted = game_vars_approximate_weighted,
#            starting_temperature = 0.4)

In [ ]:
#old wf 100: idle after quickstop frame 29192, Wafel indexing
#gets star around 29430

#new wf 100: idle after quickstop frame 28911, Wafel indexing